[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/vicente-gonzalez-ruiz/vector_quantization/blob/main/docs/spatial_color_VQ.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/vicente-gonzalez-ruiz/vector_quantization/blob/main/docs/spatial_color_VQ.ipynb)


# Vector Quantization (in the 2D domain) of a color (RGB) image

See [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans).

In [ ]:
import math

In [ ]:
try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

In [ ]:
try:
    from sklearn import cluster
except:
    !pip install sklearn
    from sklearn import cluster

In [ ]:
try:
    import gzip
except:
    !pip install gzip
    import gzip

In [ ]:
try:
    from skimage import io
except:
    !pip install scikit-image
    from skimage import io

In [ ]:
try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.axes as ax
    import pylab
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.axes as ax
    import pylab

#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
%matplotlib inline

In [ ]:
try:
    from scalar_quantization.deadzone_quantization import Deadzone_Quantizer as Quantizer                         
except:
    !pip install "scalar_quantization @ git+https://github.com/vicente-gonzalez-ruiz/scalar_quantization"
    from scalar_quantization.deadzone_quantization import Deadzone_Quantizer as Quantizer

In [ ]:
try:
    from image_IO import image_1 as gray_image
    from image_IO import image_3 as RGB_image
except:
    !pip install "image_IO @ git+https://github.com/vicente-gonzalez-ruiz/image_IO"
    from image_IO import image_1 as gray_image
    from image_IO import image_3 as RGB_image

In [ ]:
try:
    from information_theory import information
    from information_theory import distortion
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    from information_theory import information
    from information_theory import distortion

In [ ]:
try:
    from color_transforms import YCrCb as YUV
except:
    !pip install "color_transforms @ git+https://github.com/vicente-gonzalez-ruiz/color_transforms"
    from color_transforms import YCrCb as YUV

## Configuration

In [ ]:
fn = "http://www.hpca.ual.es/~vruiz/images/lena.png"
block_side = 2
block_width = block_side
block_height = block_side
N_components = 3
n_clusters = 256  # Number of bins
N_tries = 3  # Number of times K-means is run
#range_of_N_bins = range(2, 256, 1)
range_of_N_bins = [1<<i for i in range(1, 8)]
#range_of_N_bins = [1<<i for i in range(1, 10)]

#RGB_image.write = RGB_image.debug_write # Faster, but lower compression
#RGB_image.write = RGB_image.write # The fastest, but returns only an estimation of the length

## Read the image and show it

In [ ]:
#img = RGB_image.read(fn)
img = io.imread(fn)
RGB_image.show(img, fn)
print(img.shape)

## Example

In [ ]:
block_width*block_height*N_components

In [ ]:
block_length = block_width*block_height*N_components
np.random.seed(seed=1)  # makes the random numbers predictable
k_means = cluster.KMeans(init="k-means++", n_clusters=n_clusters, n_init=N_tries)
blocks = []
for i in range(0, img.shape[0], block_width):
    for j in range(0, img.shape[1], block_height):
        blocks.append(np.reshape(img[i:i + block_width, j:j + block_height], block_length))
blocks = np.asarray(blocks).astype(float)
k_means.fit(blocks)
centroids = k_means.cluster_centers_.squeeze().astype(np.uint8)  # the code-book
labels = k_means.labels_  # Labels of the centroids

labels = labels.reshape(img.shape[0]//block_height, img.shape[1]//block_width)
img_dequantized = np.empty_like(img)
for i in range(0, img.shape[0], block_width):
    for j in range(0, img.shape[1], block_height):
        img_dequantized[i:i + block_width, j:j + block_height] = centroids[labels[i//block_width,j//block_height]].reshape(block_height, block_width, N_components)

In [ ]:
RGB_image.show(img_dequantized, "Dequantized Image")
assert len(centroids) == n_clusters
print("centroids =\n", centroids)
bits_per_block = int(math.log(n_clusters)/math.log(2))
print(f"{len(centroids)} centroids ({bits_per_block} bits/block)")
blocks_in_y = img.shape[0]//block_height
blocks_in_x = img.shape[1]//block_width
print(f"{blocks_in_y}x{blocks_in_x} blocks (vectors) in the image")
number_of_blocks = blocks_in_y*blocks_in_x
total_number_of_bits = number_of_blocks*bits_per_block
print(f"total number of output bytes = {total_number_of_bits//8}")

In [ ]:
centroids.shape

In [ ]:
centroids.dtype

## RD performance

In [ ]:
def RD_curve(img, range_of_N_bins):
    blocks = []
    for i in range(0, img.shape[0], block_width):
        for j in range(0, img.shape[1], block_height):
            blocks.append(np.reshape(img[i:i + block_width, j:j + block_height], block_length))
    blocks = np.asarray(blocks).astype(float)
    points = []
    for n in range_of_N_bins:
        initial_centroids = np.ones(shape=(n, block_width*block_height*N_components))*255
        for i in range(n):
            initial_centroids[i] = np.round(initial_centroids[i]/n)
        k_means = cluster.KMeans(n_clusters=n, random_state=0)
        #k_means = cluster.KMeans(init="k-means++", n_clusters=n, n_init=N_tries, algorithm="elkan")
        #k_means = cluster.KMeans(init=initial_centroids, n_init=1, n_clusters=n, random_state=0, algorithm="elkan")
        k_means.fit(blocks)
        centroids = k_means.cluster_centers_.squeeze().astype(np.uint8)
        print(centroids.shape)
        centroids_energy = np.empty(centroids.shape[0])
        print("len(centroids_energy) =", len(centroids_energy))
        counter = 0
        for i in centroids:
            #print(".")
            centroids_energy[counter] = information.energy(i)
            #print(i, centroids_energy[counter])
            #centroids_energy[counter] = YUV.from_RGB(i.astype(np.int16))
            counter += 1
        print("centroids_energy =", centroids_energy)
        argsort_centroids = np.argsort(centroids_energy)
        print("argsort_centroids =", argsort_centroids)
        #lut = np.empty_like(argsort_centroids)
        #lut[argsort_centroids] = np.arange(n)
        #print("lut =", lut)

        k = k_means.labels_.astype(np.uint8)  # bit-depth depends on number of bins! 
        lut = np.empty_like(argsort_centroids)
        lut[np.arange(n)] = argsort_centroids
        print(centroids_energy[lut])
        #for i in range(n):
        #    print(centroids_energy[lut[i]])
        
        #k = argsort_centroids[k.copy()]
        #k = lut[k]
        k = k.reshape(img.shape[0]//block_height, img.shape[1]//block_width)
        y = np.empty_like(img)
        for i in range(0, img.shape[0], block_width):
            for j in range(0, img.shape[1], block_height):
                y[i:i + block_width, j:j + block_height] = centroids[k[i//block_width,j//block_height]].reshape(block_height, block_width, N_components)
        RGB_image.show(y, f"Reconstruction (n={n})")
        print("Quantization indexes: ", np.unique(k))
        #gray_image.show_normalized(lut[k])
        #gray_image.show_normalized(k)
        gray_image.show_normalized(centroids_energy[k], f"Centroids Energy (n={n})")
        #gray_image.show_normalized(centroids_energy[lut[k]])
        #rate = gray_image.write(lut[k], "/tmp/" + str(n) + '_', 0)*8/img.size
        rate = gray_image.write(k, "/tmp/" + str(n) + ".png")*8/img.size
        print("code-book length =", len(centroids))
        print("code-book size =", centroids.size)
        with gzip.GzipFile("/tmp/codebook.npy.gz", "w") as f:
            np.save(file=f, arr=centroids)
        #code_book_bytes = os.path.getsize("/tmp/codebook.npy.gz")
        code_book_bytes2 = RGB_image.write(centroids.reshape(n, block_width*block_height, 3), "/tmp/codebook" + str(n) + ".png")
        #print("code-book bytes =", code_book_bytes)
        print("code-book bytes2 =", code_book_bytes2)        
        #rate += code_book_bytes*8/(img.shape[0]*img.shape[1])
        rate += code_book_bytes2*8/(img.shape[0]*img.shape[1])
        _distortion = distortion.RMSE(img, y)
        if not n%10:
            plt.title(f"{n}")
            plt.imshow(y, cmap=plt.cm.gray, vmin=0, vmax=256)
        plt.show()
        points.append((rate, _distortion))
        print(f"n={n:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

In [ ]:
RD_points = RD_curve(img, range_of_N_bins)

In [ ]:
pylab.figure(dpi=150)
#pylab.scatter(*zip(*RD_points), label=f"VQ+PNG", s=1, marker='.')
pylab.plot(*zip(*RD_points), c='m', marker='x', label="PNG(VQ" + f"$^{block_width}$" + "$^x$" + f"$^{block_width}$" + ')', linestyle="dotted")
pylab.title(f"Rate/Distortion Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open(f"VQ_2D_RGB_RD_points.txt", 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
#img = RGB_image.read(fn)
if YUV.name == "YCoCg":
    img = img.astype(np.int16)
if YUV.name =="color-DCT":
    img = img.astype(float)
img = YUV.from_RGB(img)
RGB_image.show_normalized(img)

In [ ]:
block_length = block_width*block_height*N_components
np.random.seed(seed=1)  # makes the random numbers predictable
k_means = cluster.KMeans(init="k-means++", n_clusters=n_clusters, n_init=N_tries)
blocks = []
for i in range(0, img.shape[0], block_width):
    for j in range(0, img.shape[1], block_height):
        blocks.append(np.reshape(img[i:i + block_width, j:j + block_height], block_length))
blocks = np.asarray(blocks).astype(float)
k_means.fit(blocks)
centroids = k_means.cluster_centers_.squeeze().astype(np.uint16)  # the code-book
labels = k_means.labels_  # Labels of the centroids

In [ ]:
labels = labels.reshape(img.shape[0]//block_height, img.shape[1]//block_width)
img_dequantized = np.empty_like(img)
for i in range(0, img.shape[0], block_width):
    for j in range(0, img.shape[1], block_height):
        img_dequantized[i:i + block_width, j:j + block_height] = centroids[labels[i//block_width,j//block_height]].reshape(block_height, block_width, N_components)

In [ ]:
#img_dequantized = img.copy()

In [ ]:
RGB_image.show_normalized(img_dequantized)
img_dequantized = YUV.to_RGB(img_dequantized).astype(np.uint8)
RGB_image.show(img_dequantized, "Dequantized Image")
assert len(centroids) == n_clusters
print("centroids =\n", centroids)
bits_per_block = int(math.log(n_clusters)/math.log(2))
print(f"{len(centroids)} centroids ({bits_per_block} bits/block)")
blocks_in_y = img.shape[0]//block_height
blocks_in_x = img.shape[1]//block_width
print(f"{blocks_in_y}x{blocks_in_x} blocks (vectors) in the image")
number_of_blocks = blocks_in_y*blocks_in_x
total_number_of_bits = number_of_blocks*bits_per_block
print(f"total number of output bytes = {total_number_of_bits//8}")

In [ ]:
def RD_curve(img, range_of_N_bins):
    if YUV.name == "YCoCg":
        img = img.astype(np.int16)
    YUV_img = YUV.from_RGB(img)
    blocks = []
    for i in range(0, YUV_img.shape[0], block_width):
        for j in range(0, YUV_img.shape[1], block_height):
            blocks.append(np.reshape(YUV_img[i:i + block_width, j:j + block_height], block_length))
    blocks = np.asarray(blocks).astype(float)
    points = []
    for n in range_of_N_bins:
        k_means = cluster.KMeans(init="k-means++", n_clusters=n, n_init=N_tries)
        k_means.fit(blocks)
        centroids = k_means.cluster_centers_.squeeze().astype(np.uint16)
        k = k_means.labels_.astype(np.uint16)  # bit-depth depends on number of bins! 
        k = k.reshape(YUV_img.shape[0]//block_height, YUV_img.shape[1]//block_width)
        YUV_y = np.empty_like(YUV_img)
        for i in range(0, YUV_img.shape[0], block_width):
            for j in range(0, YUV_img.shape[1], block_height):
                YUV_y[i:i + block_width, j:j + block_height] = centroids[k[i//block_width,j//block_height]].reshape(block_height, block_width, N_components)
        print("Quantization indexes: ", np.unique(k))
        rate = gray_image.write(k, "/tmp/" + str(n) + ".png")*8/img.size
        y = YUV.to_RGB(YUV_y).astype(np.uint8)
        _distortion = distortion.RMSE(img, y)
        RGB_image.show(y)
        if not n%10:
            plt.title(f"{n}")
            plt.imshow(y, cmap=plt.cm.gray, vmin=0, vmax=256)
            plt.show()
        points.append((rate, _distortion))
        print(f"n={n:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

In [ ]:
#img = RGB_image.read(fn)
RD_points_YUV = RD_curve(img, range_of_N_bins)

In [ ]:
pylab.figure(dpi=150)
#pylab.scatter(*zip(*RD_points), label=f"VQ+PNG", s=1, marker='.')
pylab.plot(*zip(*RD_points), label=f"VQ(2D+RGB)+PNG", linestyle="dotted")
pylab.plot(*zip(*RD_points_YUV), label=f"VQ(2D+{YUV.name})+PNG", linestyle="dotted")
pylab.title(f"Rate/Distortion Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

## Conclusions

1. VQ can remove at the same time both, the color and the spatial redundancy.

In [ ]:
import time
while True:
    time.sleep(1)